# **Multi-output Regression Use Case Demonstration**

##### **Author: Partha Seetala**

**Video Tutorial: https://youtu.be/CQTCS8SO8bs**


**Given the `sqft` and `lotsize` (2 input features) predict the `price` and `bedrooms` (2 output features) of a house**

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from keras.initializers import he_normal
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [ ]:
# This function generates some synthetic data {sqft, bedrooms, lotsize and price}
def generate_data(count, noutputs=1):
    sqft_ranges = [0, 1500, 2500, 3500, 10000]
    price_ranges = [800000, 1000000, 2500000, 3000000]

    x = []
    y = []

    for i in range(count):
        sqft = random.randint(1800,4500)
        nbedrooms = 1+ int(sqft/1000)
        lotsize = int(random.uniform(1.2, 2.0) * sqft)

        price = 0
        for r in range(len(sqft_ranges)-1):
            if sqft >= sqft_ranges[r] and sqft < sqft_ranges[r+1]:
                price = price_ranges[r]
                break
        assert price != 0

        lotsize_value = int(random.randint(50000, 300000) * (lotsize/sqft))
        price = price + lotsize_value
        price = round(price, -3)

        if noutputs == 1:
            x.append([sqft, nbedrooms, lotsize])
            y.append(price)
        else:
            x.append([sqft, lotsize])
            y.append([price, nbedrooms])

    return x, y

**Let's initialize a Standardization module we'll use in our model**

In [ ]:
norm_x = StandardScaler() # We'll use this to standardize our input
norm_y = StandardScaler() # We'll use this to standardize our output

**Let's `build` a model to predict 2 outputs**

In [ ]:
def build_model_for_two_outputs():
    model = Sequential()

    # Hidden Layer(s)
    model.add(Dense(10, input_dim=2, kernel_initializer=he_normal(), activation="relu"))

    # Output Layer
    model.add(Dense(2))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

model2 = build_model_for_two_outputs()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Let's generate some data**

In [ ]:

# Generate some house price data
x, y = generate_data(10000, noutputs=2)

# Let's display the data we generated
data = pd.DataFrame(
    {'X_sqft': [rec[0] for rec in x],
    'X_lotsize': [rec[1] for rec in x],
    'Y_price': [rec[0] for rec in y],
    'Y_nbedrooms': [rec[1] for rec in y]})
data

,X_sqft,X_lotsize,Y_price,Y_nbedrooms
0,3665,5730,3089000,4
1,2006,3945,1276000,3
2,2111,3847,1170000,3
3,3437,6046,2697000,4
4,2185,4092,1359000,3
...,...,...,...,...
9995,3842,7548,3107000,4
9996,4248,7982,3243000,5
9997,3036,4995,2926000,4
9998,4297,7552,3181000,5


**Let's `train` our model**

In [ ]:
def train_model_for_two_outputs(model, x, y):
    # Normalize data
    xnorm = norm_x.fit_transform(np.array(x))
    ynorm = norm_y.fit_transform(np.array(y))

    # Train the model
    model.fit(xnorm, ynorm, epochs=100, batch_size=10, verbose=1)

train_model_for_two_outputs(model2, x, y)

Epoch 1/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.2104
Epoch 2/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1283
Epoch 3/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1129
Epoch 4/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0884
Epoch 5/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0760
Epoch 6/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0711
Epoch 7/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0707
Epoch 8/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0681
Epoch 9/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0686
Epoch 10/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0697
Epoch 11/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0697
Epoch 12/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0681
Epoch 13/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0673
Epoch 14/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0681
Epoch 15/100
10

**Let's use our model to make predictions**

In [ ]:
def predict_model_for_two_outputs(model):
    x, yt = generate_data(20, noutputs=2)

    xnorm = norm_x.transform(np.array(x))

    predictions = model.predict(xnorm)

    # De-Normalize predictions
    yp = norm_y.inverse_transform(predictions)

    return x, yt, yp

x, yt, yp = predict_model_for_two_outputs(model2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


**Let's see what our predictions look like**

In [ ]:
def show_predictions_for_two_outputs(x, yt, yp):
    price_offby = []
    price_percent = []

    nbedrooms_offby = []
    nbedrooms_percent = []

    for i in range(len(yp)):
        price_diff = int(yt[i][0] - yp[i][0])
        price_offby.append(price_diff)
        price_percent.append(int(100*(price_diff/yt[i][0])))

        nbedrooms_diff = yt[i][1] - yp[i][1]
        nbedrooms_offby.append(nbedrooms_diff)
        nbedrooms_percent.append(int(100*(nbedrooms_diff/yt[i][1])))

    data = pd.DataFrame(
            {'sqft': [rec[0] for rec in x],
            'lotsize': [rec[1] for rec in x],
            'price': [int(p[0]) for p in yt],
            'price_pred': [int(p[0]) for p in yp],
            'price_offby': price_offby,
            'price_percent_off': price_percent,
            'nbedrooms': [p[1] for p in yt],
            'nbedrooms_pred': [int(p[1]) for p in yp],
            'nbedrooms_offby': nbedrooms_offby,
            'nbedrooms_percent_of': nbedrooms_percent})
    return data

show_predictions_for_two_outputs(x, yt, yp)

,sqft,lotsize,price,price_pred,price_offby,price_percent_off,nbedrooms,nbedrooms_pred,nbedrooms_offby,nbedrooms_percent_of
0,3101,6018,2778000,2849832,-71832,-2,4,3,0.226140,5
1,2406,4773,1438000,1336266,101734,7,3,3,-0.214470,-7
2,3043,5680,2831000,2835995,-4995,0,4,3,0.337911,8
3,2226,3233,1403000,1236711,166288,11,3,2,0.119296,3
4,2865,4372,2935000,2776917,158083,5,3,3,-0.316411,-10
5,2852,4406,2798000,2780552,17447,0,3,3,-0.292435,-9
6,2461,4062,1467000,1494125,-27125,-1,3,3,-0.207785,-6
7,2511,3352,2657000,2092310,564689,21,3,2,0.012777,0
8,2570,4603,2876000,2826234,49765,1,3,2,0.232944,7
9,2851,5608,2898000,2854446,43553,1,3,3,-0.302361,-10
